In [12]:
import pandas as pd
import tensorflow as tf
import numpy as np

DROPOUT=0.5

TRAINING_ITERATIONS =10000
LEARNING_RATE = 1e-4
Validation_size=2000
BATCH_SIZE=50




In [13]:
data=pd.read_csv('~/Documents/GIT_HUB/MNIST/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
images=data.iloc[:,1:].values.astype(np.float);
images=np.multiply(images,1.0/255.0);
#this is done to change the shape of the array
print(images.shape);
image_size=images.shape[1]
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
print(image_width)

(32000, 784)
28


In [15]:
labels_count=data[[0]].values.ravel()
label = np.unique(labels_count).shape[0]
print(label);

10


In [16]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    res=np.zeros(shape=(num_labels,num_classes));
    for i in range (0,num_labels):
        res[i,labels_dense[i]]=1;
    return res;

print(labels_count.shape);
hot_label=dense_to_one_hot(labels_count,label);
hot_label = hot_label.astype(np.uint8)
print(hot_label[10])
print(labels_count[10])

(32000,)
[0 0 0 0 0 0 0 0 1 0]
8


In [17]:
validation_images=images[:Validation_size]
validation_labels=hot_label[:Validation_size]

train_images=images[Validation_size:]
train_labels=hot_label[Validation_size:]
print(validation_images.shape)
print(train_images.shape)

(2000, 784)
(30000, 784)


In [18]:
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [19]:
x = tf.placeholder('float', shape=[None, image_size])
#W=tf.placeholder('float',shape=[images.shape[1],10])
y_=tf.placeholder('float',shape=[None,label])

In [20]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

image = tf.reshape(x, [-1,image_width , image_height,1])


h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)
#print (h_conv1.get_shape()) # => (40000, 28, 28, 32)
h_pool1 = max_pool_2x2(h_conv1)
#print (h_pool1.get_shape()) # => (40000, 14, 14, 32)



# second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)



W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# (40000, 7, 7, 64) => (40000, 3136)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


#this is the output layer
W_fc2 = weight_variable([1024, label])
b_fc2 = bias_variable([label])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


# optimisation function
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# evaluation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
predict = tf.argmax(y,1)

In [21]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f111b68da58>>
Traceback (most recent call last):
  File "/home/karan/anaconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 178, in __del__
    self.close()
  File "/home/karan/anaconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1028, in close
    self._default_session.__exit__(None, None, None)
  File "/home/karan/anaconda3/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/home/karan/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3529, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


In [22]:
# visualisation variables
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(50)        
    if(i%100==0):
        print(i)
        print(sess.run(accuracy,feed_dict={x:batch_xs,y_:batch_ys,keep_prob:1.0}))
        if(Validation_size):#this is done to make sure that in case we make validation size 0 , we can simply avoid printing it
            print(sess.run(accuracy,feed_dict={x:validation_images,y_:validation_labels,keep_prob:1.0}))
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

0
0.22
0.1455
100
0.84
0.8415
200
0.92
0.8935
300
0.92
0.9115
400
0.9
0.9335
500
0.98
0.9365
600
0.96
0.949
700
0.94
0.948
800
0.92
0.9505
900
0.94
0.96
1000
0.94
0.959
1100
0.98
0.964
1200
0.94
0.961
1300
0.94
0.967
1400
1.0
0.967
1500
1.0
0.9695
1600
0.98
0.969
1700
1.0
0.974
1800
0.98
0.9725
1900
0.96
0.973
2000
0.96
0.973
2100
0.94
0.971
2200
0.96
0.975
2300
0.98
0.9785
2400
0.98
0.9825
2500
0.98
0.981
2600
1.0
0.981
2700
0.98
0.982
2800
0.96
0.983
2900
1.0
0.984
3000
0.98
0.979
3100
0.98
0.9825
3200
1.0
0.983
3300
0.98
0.9815
3400
0.98
0.9825
3500
1.0
0.9855
3600
1.0
0.9835
3700
1.0
0.9875
3800
0.98
0.988
3900
0.96
0.9875
4000
1.0
0.983
4100
0.96
0.985
4200
1.0
0.9845
4300
0.96
0.985
4400
0.98
0.9885
4500
1.0
0.985
4600
1.0
0.986
4700
0.98
0.984
4800
1.0
0.9875
4900
1.0
0.9845
5000
1.0
0.9855
5100
1.0
0.9875
5200
1.0
0.989
5300
1.0
0.987
5400
1.0
0.989
5500
1.0
0.985
5600
1.0
0.989
5700
0.96
0.985
5800
1.0
0.9875
5900
0.98
0.9885
6000
1.0
0.987
6100
1.0
0.9875
6200
1.0
0.9885
6300

In [38]:
test=pd.read_csv('~/Documents/GIT_HUB/MNIST/test.csv').values
test=test.astype(np.float);
test=np.multiply(test,1.0/255.0);
predicted_values=np.zeros(shape=(test.shape[0]));
for i in range(0,test.shape[0]//BATCH_SIZE):
    predicted_values[i*(BATCH_SIZE):(i+1)*BATCH_SIZE]=predict.eval(feed_dict={x:test[(i*BATCH_SIZE):(i+1)*BATCH_SIZE],keep_prob:1.0});
print(predicted_values[10])
    

1.0


In [40]:
np.savetxt('submission.csv', 
           np.c_[range(1,len(test)+1),predicted_values], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')